In [19]:
import numpy as np
import pandas as pd
import random
from sqlalchemy.engine import create_engine
from mysql.connector import connect
import mysql.connector
from mysql.connector import Error
import pymysql
import warnings
import traceback
from pandas.io import sql
from sqlalchemy import create_engine

from random import randrange
import datetime
from datetime import timedelta
from sklearn import preprocessing
# import datetime

# import MySQLdb

warnings.filterwarnings("ignore")

In [20]:
from urllib.parse import quote_plus


In [21]:
# connect db
def connect_db():
    user_name = "root"
    pwd = "Trung99"
    server = "localhost"
    db = "tastie"
    conn = connect(host=server, user=user_name, password=pwd, database=db, port=3306)
    cur = conn.cursor(buffered=True)
    return conn, cur


# conn = create_engine(conn_str)

In [22]:
def close_connection(cur, conn):
    cur.close()
    conn.close()

In [23]:
def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)


In [24]:
def insert_execute_many(dataset, table_name):
    conn, cur = connect_db()

    cols = "`,`".join([str(i) for i in list(dataset)])
    sql = (
        "INSERT INTO `"
        + table_name
        + "` (`"
        + cols
        + "`) VALUES ("
        + "%s," * (dataset.shape[1] - 1)
        + "%s)"
    )

    try:
        cur.executemany(sql, dataset.values.tolist())
        conn.commit()
    except Error as error:
        conn.rollback()
        print("Failed to insert into MySQL table {}".format(error))
    finally:
        cur.close()
        conn.close()

In [25]:
# for dataset > 100.000 records and many columns (ex: product)
def insert_execute_many_large_dataset(dataset, table_name):
    conn, cur = connect_db()
    dataset_size = len(dataset)
    step = 100000

    for start in range(0, len(dataset), step):
        if dataset_size < step:
            step = abs(step-dataset_size)
            end = start + step
            insert_execute_many(dataset[start:end], table_name)
            break
        end = start + step
        insert_execute_many(dataset[start:end], table_name) 

        dataset_size -= step


### User

In [26]:
df_user = pd.read_csv("mysql_data/user.csv")

In [27]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39128 entries, 0 to 39127
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   user_id        39128 non-null  int64  
 1   first_name     39128 non-null  object 
 2   last_name      39128 non-null  object 
 3   gender         39128 non-null  int64  
 4   birthday       39128 non-null  object 
 5   email          39128 non-null  object 
 6   phone          39128 non-null  int64  
 7   password       39128 non-null  object 
 8   role           39128 non-null  int64  
 9   user_token     39128 non-null  object 
 10  registered_at  39128 non-null  object 
 11  update_at      39128 non-null  object 
 12  delete_at      0 non-null      float64
 13  avatar         0 non-null      float64
dtypes: float64(2), int64(4), object(8)
memory usage: 4.2+ MB


In [28]:
df_user.delete_at = (
    df_user.delete_at.fillna(np.nan).replace([np.nan], [None])
)

df_user.avatar = (
    df_user.avatar.fillna(np.nan).replace([np.nan], [None])
)

In [29]:
insert_execute_many(df_user, 'user')

### Cart

In [30]:
df_cart = pd.read_csv("mysql_data/cart.csv")

In [31]:
insert_execute_many(df_cart, 'cart')

### Owner

In [32]:
df_owner = pd.read_csv("mysql_data/owner.csv")

In [33]:
#df_owner.info()

df_owner.delete_at = (
    df_owner.delete_at.fillna(np.nan).replace([np.nan], [None])
)

In [34]:
insert_execute_many(df_owner, 'owner')

### Provider

In [35]:
df_provider = pd.read_csv("mysql_data/provider.csv")

In [36]:
# df_provider.info()

df_provider.delete_at = (
    df_provider.delete_at.fillna(np.nan).replace([np.nan], [None])
)

df_provider.description = (
    df_provider.description.fillna(np.nan).replace([np.nan], [None])
)

In [37]:
insert_execute_many(df_provider, 'provider')

### Provider Operation

In [38]:
df_provider_operation = pd.read_csv("mysql_data/provider_operation.csv")

In [39]:
#df_provider_operation.info()
df_provider_operation.delete_at = (
    df_provider_operation.delete_at.fillna(np.nan).replace([np.nan], [None])
)

In [40]:
insert_execute_many_large_dataset(df_provider_operation, 'operation')

### Product

In [41]:
df_product = pd.read_csv("mysql_data/product.csv")

In [42]:
# df_product.info()

df_product.delete_at = (
    df_product.delete_at.fillna(np.nan).replace([np.nan], [None])
)

df_product.description = (
    df_product.description.fillna(np.nan).replace([np.nan], [None])
)

df_product.product_image = (
    df_product.product_image.fillna(np.nan).replace([np.nan], [None])
)

In [43]:
insert_execute_many_large_dataset(df_product, 'product')

### Product Option

In [44]:
df_product_option = pd.read_csv("mysql_data/productoption.csv")

In [45]:
df_product_option.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12921 entries, 0 to 12920
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_id          12921 non-null  int64  
 1   label               12921 non-null  object 
 2   value               12921 non-null  object 
 3   option_description  0 non-null      float64
 4   is_required         12921 non-null  int64  
 5   price               12921 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 605.8+ KB


In [46]:
df_product_option.option_description = None
df_product_option.head()

,product_id,label,value,option_description,is_required,price
0,1000000,?,?,None,1,0
1,1000001,?,?,None,1,0
2,1000002,?,?,None,1,0
3,1000003,?,?,None,1,0
4,1000004,?,?,None,1,0


In [47]:
insert_execute_many_large_dataset(df_product_option, 'productoption')

### Provider Category

In [48]:
df_provider_category = pd.read_csv("mysql_data/provider_category.csv")

In [49]:
insert_execute_many(df_provider_category, 'ProviderCategory')

### Provider Category Detail

In [50]:
df_provider_category_detail = pd.read_csv("mysql_data/provider_category_detail.csv")

In [51]:
insert_execute_many_large_dataset(df_provider_category_detail, 'ProviderCategoryDetail')

### Cuisine Category

In [52]:
df_cuisine_category = pd.read_csv("mysql_data/cuisine_category.csv")

In [53]:
insert_execute_many_large_dataset(df_cuisine_category, 'CuisineCategory')

### Cuisine Category Detail

In [54]:
df_cuisine_category_detail = pd.read_csv("mysql_data/cuisine_category_detail.csv")

In [55]:
insert_execute_many_large_dataset(df_cuisine_category_detail, 'CuisineCategoryDetail')

### Main Food Category

In [56]:
df_main_food_category = pd.read_csv("mysql_data/main_food_category.csv")

In [57]:
insert_execute_many(df_main_food_category, 'MainFoodCategory')

### Main Food Category Detail

In [58]:
df_main_food_category_detail = pd.read_csv("mysql_data/main_food_category_detail.csv")

In [59]:
insert_execute_many_large_dataset(df_main_food_category_detail, 'MainFoodCategoryDetail')

### Food Category

In [60]:
df_food_category = pd.read_csv("mysql_data/food_category.csv")

In [61]:
insert_execute_many(df_food_category, 'FoodCategory')

### Food Category Detail

In [62]:
df_food_category_detail = pd.read_csv("mysql_data/food_category_detail.csv")

In [63]:
insert_execute_many_large_dataset(df_food_category_detail, 'FoodCategoryDetail')

### Menu Category

In [64]:
df_menu_category = pd.read_csv("mysql_data/menu_category.csv")

In [65]:
# df_menu_category.info()

In [66]:
insert_execute_many(df_menu_category, 'MenuCategory')

### Menu Category Detail

In [67]:
df_menu_category_detail = pd.read_csv("mysql_data/menu_category_detail.csv")

In [68]:
insert_execute_many_large_dataset(df_menu_category_detail, 'MenuCategoryDetail')